In [64]:
from bokeh.io import reset_output

reset_output()

In [65]:
from bokeh.plotting import figure, show, output_server

p = figure(title="Server Plot")
p.circle([1, 2, 3], [4, 5, 6])

output_server("hover")

show(p)

In [68]:
import numpy as np
from numpy import pi

from bokeh.client import push_session
from bokeh.driving import cosine
from bokeh.plotting import figure, curdoc

reset_output()

x = np.linspace(0, 4*pi, 80)
y = np.sin(x)

p = figure()
r0 = p.line([0, 4*pi], [-1, 1], color="firebrick")
r1 = p.line(x, y, color="navy", line_width=4) # A sine curve

# open a session and push document to server, server and client will be in sync
session = push_session(curdoc(),session_id='sine')

@cosine(w=0.03) #a driver function that advances a sequence of cosine values (w=0.03 is frequency)
                # value = cos(w*i)
def update(step):
    r1.data_source.data["y"] = y * step # multiply sine values with cosine value from update
    r1.glyph.line_alpha = 1 - 0.8 * abs(step)

# Add a callback to be invoked on a session periodically (here frequency = 50)
curdoc().add_periodic_callback(update, 50)

session.show() # open the document in a browser

session.loop_until_closed() # run forever

In [ ]:
import numpy as np
from bokeh.client import push_session
from bokeh.models import Button
from bokeh.palettes import RdYlBu3
from bokeh.plotting import figure, curdoc, vplot
from bokeh.io import reset_output

reset_output()

# open a session: while the connection to the server is open, changes made on the server side will be applied document/plot
# curdoc() returns the document for the current default state
session = push_session(curdoc(),session_id='button')

# create a plot and style its properties
p = figure(x_range=(0, 100), y_range=(0, 100), toolbar_location=None)
p.border_fill_color = 'black'
p.background_fill_color = 'black'

# add a text renderer to out plot (no data yet)
r = p.text(x=[], y=[], text=[], text_color=[], text_font_size="20pt",
           text_baseline="middle", text_align="center")

i = 0 # thats were the number starts
ds = r.data_source # data source object

# create a callback that will add a number in a random location
def callback():
    global i # modify global copy if i
    ds.data['x'].append(np.random.random()*70 + 15)
    ds.data['y'].append(np.random.random()*70 + 15)
    ds.data['text_color'].append(RdYlBu3[i%3])
    ds.data['text'].append(str(i))
    ds.trigger('data', ds.data, ds.data) # now change the data attribute of the source
    i = i + 1

# add a button widget and configure with the call back
button = Button(label="Press Me")
button.on_click(callback)

# put the button and plot in a vertical layout and add to the document
curdoc().add_root(vplot(button, p))
session.show()
session.loop_until_closed() # run forever